In [1]:
import duckdb
import pandas as pd  
import os   

In [2]:
#se o banco estiver aberto fechar antes de conectar
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [26]:
# Exemplo de consulta com janela para numerar linhas por partição
#ROW NUMBER serve para identificar a ordem dos registros dentro de cada partição definida por NATBR
#PARITION BY NATBR agrupa os registros por NATBR
#NATBR É um código que identifica a natureza do produto
#NATBR ORDER BY data_ingestao DESC ordena os registros dentro de cada partição por data_ingestao em ordem decrescente

df = con.execute('''
SELECT * FROM (
SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row 
FROM bronze_z0019 
WHERE data_ingestao >= '2025-01-01')
WHERE row = 1
''').fetchdf() 

df.head(10)


,NATBR,MAKTX,WERKS,MAINST,LABST,nome_arquivo,data_ingestao,row
0,10002,MARTELO,BT50,100,1500,z0019_1.csv,2026-01-09,1
1,10003,PREGO,BT10,100,50,z0019_1.csv,2026-01-09,1
2,10004,SERRA,BT50,100,200,z0019_2.csv,2026-01-09,1
3,10005,MACHADDO,BT50,100,100,z0019_2.csv,2026-01-09,1
4,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-01-09,1


In [ ]:
df_final = df.drop(columns=['nome_arquivo','data_ingestao','row'])
df_final = df_final.rename(columns={'NATBR':'id'})
df_final = df_final.rename(columns={'MAKTX':'nm_produto'})
df_final = df_final.rename(columns={'WERKS':'id_categoria'})
df_final = df_final.rename(columns={'MAINST':'id_fornecedor'})
df_final = df_final.rename(columns={'LABST':'vl_preco'})

df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500
1,10003,PREGO,BT10,100,50
2,10004,SERRA,BT50,100,200
3,10005,MACHADDO,BT50,100,100
4,10001,PARAFUSO,BT10,100,100


In [30]:
df2.dtypes

id               object
nm_produto       object
id_categoria     object
id_fornecedor    object
vl_preco         object
dtype: object

In [34]:
df2 = df_final
df2 = df2.astype(
      {
         'id': 'int64',
         'nm_produto': 'string',
         'id_categoria': 'string',
         'id_fornecedor': 'int64',
         'vl_preco': 'float64'
      }
)

In [35]:
df2.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500.0
1,10003,PREGO,BT10,100,50.0
2,10004,SERRA,BT50,100,200.0
3,10005,MACHADDO,BT50,100,100.0
4,10001,PARAFUSO,BT10,100,100.0


In [40]:
con.execute('''
CREATE TABLE IF NOT EXISTS produtos (
      id BIGINT, 
      nm_produto TEXT,
      id_categoria TEXT,
      id_fornecedor BIGINT,
      vl_preco FLOAT
   )
   '''
)

In [48]:
#colocar os dados de df2 no banco de dados na tabela produtos
con.execute ("INSERT INTO produtos SELECT * FROM df2") #toda vez que rodar esse comando, novos dados serão inseridos na tabela produtos

In [50]:
df_resultado = con.execute('SELECT * FROM produtos').fetchdf() # busca os dados da tabela produtos

#apagar da primary key 6 em diante
df_resultado = df_resultado.drop_duplicates()


df_resultado.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500.0
1,10003,PREGO,BT10,100,50.0
2,10004,SERRA,BT50,100,200.0
3,10005,MACHADDO,BT50,100,100.0
4,10001,PARAFUSO,BT10,100,100.0


In [51]:
con.close()